In [ ]:
# from huggingface_hub import snapshot_download

# local_dir = "./dog"
# snapshot_download(
#     "diffusers/dog-example",
#     local_dir=local_dir, repo_type="dataset",
#     ignore_patterns=".gitattributes",
# )

In [ ]:
# single inference 

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "/home/paper/diffusers/examples/dreambooth/trained_ckp/starbucks-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of starbucks mug on a table"
# prompt = "A Starbucks mug sits on a table, surrounded by a cozy atmosphere and the aroma of freshly brewed coffee."
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# image.save("dog-bucket.png")
image.save("./inference/starbucks/starbuck-2-1_on_table_0.png")

# multi prompts inference

In [1]:
from diffusers import StableDiffusionPipeline
import torch
from tqdm import tqdm

model_id = "/home/paper/diffusers/examples/dreambooth/trained_ckp/lindanx"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompts = ["A photo of lindanx playing basketball.",
           "A photo of lindanx walking along the street.",
        ]
# prompts = ["A Starbucks mug sits on a table, surrounded by a cozy atmosphere and the aroma of freshly brewed coffee.",
#            "On a table, a Starbucks mug is placed, exuding a delightful coffee fragrance that fills the air.",
#            "This image showcases a Starbucks mug resting on a cute table, with its rich aroma creating a sense of tranquility.",
#            "A Starbucks mug finds repose on a table, adorned with intricate patterns and vibrant colors, making it an eye-catching centerpiece.",
#            "A steaming Starbucks coffee cup is placed on a charming table, radiating warmth and energizing vibes.",
#            "In this heartwarming scene, a Starbucks mug graces a table, offering a moment of pure enjoyment.",
#            "The Starbucks mug on the table emits a strong coffee aroma, accompanied by warm lighting, creating a delightful ambiance.",
#            "In this captivating setting, a Starbucks mug calmly rests on a table, providing a moment of comfort and pleasure.",
#            "A Starbucks mug is delicately placed on a table, blending intricate textures and soft hues to create a visually pleasing experience.",
#            "This photo showcases a Starbucks mug placed on a table, forming a vibrant scene filled with coziness and warmth."]

for i, prompt in tqdm(enumerate(prompts)):
    for infer in range(5):
        image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
        image.save("./inference/lindan/meixi_win_p{}_i{}.png".format(i, infer))
# image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# image.save("dog-bucket.png")
# image.save("./inference/starbucks/starbuck_on_table_2.png")

0it [00:00, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

2it [00:48, 24.04s/it]


In [ ]:
import cv2
import os
video_path = "/home/paper/AIGC/result/background_beach.mp4"
try:
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    i = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame,(1280, 720))
            cv2.imwrite("/home/paper/AIGC/result/interview-background/{:08d}.png".format(i), frame)
            i += 1
        else:
            break
except (OSError, TypeError, ValueError, KeyError, SyntaxError) as e:
    print("read_frame_source:{} error. {}\n".format(video_path, str(e)))

图片转视频

In [ ]:
import os
import torch
import torchvision
import numpy as np
from PIL import Image

frame_names = os.listdir("./ref_videos/guanggao_edit/")
frame_names.sort()

frames_path = [os.path.join("./ref_videos/guanggao_edit/", frame) for frame in frame_names[:34]]
frames = []
output_path = "/home/paper/AIGC/test_sample/guanggao_edit.mp4"
for frame in frames_path:
    frames.append(np.asarray(Image.open(frame)))
frames = torch.from_numpy(np.asarray(frames))
if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))
torchvision.io.write_video(output_path, frames, fps=5, video_codec="libx264")


In [ ]:
import cv2
import numpy as np

# 读取原始图像和修改后的图像
original_image = cv2.imread("./ref_videos/guanggao/00000001.png")
modified_image = cv2.imread("./ref_videos/guanggao_edit/00000001.png")

# 将图像转换为NumPy数组
original_array = np.array(original_image)
# origin_size = (original_array.shape[1], original_array.shape[0])
# modified_image= cv2.resize(modified_image, origin_size)
modified_array = np.array(modified_image)

# 计算差异
difference_array = np.abs(original_array - modified_array)

# 设定阈值
threshold = 50

# 根据阈值创建掩码
mask = np.where(difference_array > threshold, 255, 0).astype(np.uint8)
# cv2.imwrite("./ref_videos/guanggao_edit/mask.png", mask)
mask = cv2.imread("./ref_videos/guanggao_edit/mask.png", -1)

# painted_mask = cv2.bitwise_and(modified_image, modified_image, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))
painted_mask = cv2.bitwise_and(modified_image, modified_image, mask=mask)

cv2.imwrite("./ref_videos/guanggao_edit/patined_mask_01.png", painted_mask)

In [ ]:
import cv2
import os
from tqdm import tqdm
new_mask = cv2.imread("/home/paper/AIGC/test_sample/xiaohuangren_mask.png", -1)
origin_mask = cv2.imread("/home/paper/AIGC/test_sample/dogmug_mask.png", -1)

# merge two masks
mask = cv2.bitwise_or(new_mask, origin_mask)

origin_images = os.listdir("./ref_videos/guanggao/")
origin_images.sort()
origin_images_path = [os.path.join("./ref_videos/guanggao/", image) for image in origin_images]
# 读取原始图像和覆盖图像
modified_image = cv2.imread("/home/paper/AIGC/test_sample/guanggao_xiaohuangren_resize.png")
for i, image in tqdm(enumerate(origin_images_path)):

    original_image = cv2.imread(image)
    
    # original_image = cv2.imread("./ref_videos/guanggao/00000002.png")
    foreground = mask==0

    # 将原始图像中的对应区域替换为另一张图像
    # result = cv2.bitwise_and(original_image, original_image, mask=mask)
    # replacement_image = cv2.imread("replacement.jpg")

    mask_color = cv2.bitwise_and(modified_image, modified_image, mask=mask)
    origin_foreground = cv2.bitwise_and(original_image, original_image, mask=foreground.astype('uint8'))


    result = cv2.bitwise_or(origin_foreground, mask_color)



    # 显示结果
    cv2.imwrite("./ref_videos/guanggao_xiaohuangren/{:08d}.png".format(i+1), result)
    cv2.imwrite("/home/paper/AIGC/result/gt_mask/xiaohuangren_merge/{:08d}.png".format(i+1), result)



In [ ]:
# background change
import cv2
import os
from tqdm import tqdm

origin_images = os.listdir("/home/paper/AIGC/result/originimages/kunkun_interview.mp4/")
origin_images.sort()
origin_images_path = [os.path.join("/home/paper/AIGC/result/originimages/kunkun_interview.mp4/", image) for image in origin_images]

masks = os.listdir("/home/paper/AIGC/result/gt_mask/kunkun_interview/")
masks.sort()
masks_path = [os.path.join("/home/paper/AIGC/result/gt_mask/kunkun_interview/", mask) for mask in masks]
# 读取原始图像和覆盖图像


modified_images = os.listdir("/home/paper/AIGC/result/interview-background/")
modified_images.sort()
modified_images_path = [os.path.join("/home/paper/AIGC/result/interview-background/", image) for image in modified_images]
background_id = 0
for i, image in tqdm(enumerate(origin_images_path)):
    mask = cv2.imread(masks_path[i],-1)
    original_image = cv2.imread(image)
    modified_image = cv2.imread(modified_images_path[background_id])
    # original_image = cv2.imread("./ref_videos/guanggao/00000002.png")
    background = mask==0

    # 将原始图像中的对应区域替换为另一张图像
    # result = cv2.bitwise_and(original_image, original_image, mask=mask)
    # replacement_image = cv2.imread("replacement.jpg")

    mask_color = cv2.bitwise_and(modified_image, modified_image, mask=background.astype('uint8'))
    origin_foreground = cv2.bitwise_and(original_image, original_image, mask=mask)


    result = cv2.bitwise_or(origin_foreground, mask_color)
    if background_id < len(modified_images_path)-1:
        background_id+=1
    else:
        background_id = 0


    # 显示结果
    cv2.imwrite("/home/paper/AIGC/result/kunkun_background/{:08d}.png".format(i+1), result)

In [ ]:



# 显示结果
cv2.imwrite("./ref_videos/guanggao_edit/00000002_foreground.png", origin_foreground)

In [ ]:

cv2.imwrite("./ref_videos/guanggao_edit/00000002.png", result)